In [ ]:
import pandas as pd
from itertools import takewhile
import plotly.graph_objects as go
import base64
import numpy as np

from dare_viz import path

In [ ]:
def read_intem(species, year):

    file_str = {2021: "Table_UK_2021_SPECIES.txt",
                2023: "best_intem_UK_SPECIES_AR2023.csv",}

    fname = path() / f"data/intem_{year}/{file_str[year].replace("SPECIES", species)}"

    with open(fname, "r") as f:
        headiter = takewhile(lambda s: s.startswith('#'), f)
        header = list(headiter)

    df = pd.read_csv(fname,
                    sep=",",
                    skipinitialspace=True,
                    comment="#")

    # remove whitespace from headers
    df.columns = df.columns.str.replace(" ", "")

    # Change all columns to lower case
    df.columns = map(str.lower, df.columns)

    # Remove any dates from column names
    column_names = []
    for col in df.columns:
        if "20" in col:
            column_names.append(col.split("20")[0])
        else:
            column_names.append(col)
    df.columns = column_names

    # Rename "inventory" to "invent", if present
    if "inventory" in df.columns:
        df = df.rename(columns={"inventory": "invent"})

    # Set index to year
    df = df.set_index('year')

    df = df.apply(pd.to_numeric, errors='ignore')
    df.index = pd.to_datetime(df.index, format='%Y')

    for h in header:
        if "GAS" in h:
            species_str = h.split(":")[1].strip()
        if "UNITS" in h:
            species_units = h.split(":")[1].strip()
        if "GWP" in h:
            species_gwp = float(h.split(":")[1].strip())
    
    return df, species_str, species_units, species_gwp

In [ ]:
intem_year = 2023

# Get F-gases
fgas = [f"hfc{hfc}" for hfc in ["23", "32", "125", "134a", "143a", "152a", "227ea", "245fa", "365mfc", "4310mee"]] + \
    [f"pfc{pfc}" for pfc in ["14", "116", "218", "318"]] + \
    ["sf6"]

df_fgas, _, _, _ = read_intem(fgas[0], intem_year)

uncert = df_fgas["intem_std"]**2

for fg in fgas[1:]:
    dfs, _, _, _ = read_intem(fg, intem_year)
    df_fgas += dfs
    uncert += df_fgas["intem_std"]**2

df_fgas["intem_std"] = np.sqrt(uncert.values)

In [ ]:
df_ch4, _, _, _ = read_intem("ch4", intem_year)
df_n2o, _, _, _ = read_intem("n2o", intem_year)

In [ ]:
alpha = 0.6
color_ch4 = ["rgba(86, 119, 194, 1)", f"rgba(86, 119, 194, {alpha})"]
color_n2o = ["rgba(99, 182, 137, 1)", f"rgba(99, 182, 137, {alpha})"]
color_fgas = ["rgba(187, 103, 93, 1)", f"rgba(187, 103, 93, {alpha})"]

fig = go.Figure()

mo_logo = base64.b64encode(open("metoffice_logo.png", 'rb').read())
uob_logo = base64.b64encode(open("uob_logo.png", 'rb').read())

def errorbars(df, var, color, dash, name, showlegend=True):

    error_minus=df[var] - df[f"{var}_std"]
    error_plus=df[var] + df[f"{var}_std"]

    fig.add_trace(go.Scatter(
        x=df.index,
        y=error_minus.values,
        fill=None,
        mode='lines',
        line=dict(color=color[1], width=0.1),
        showlegend=False,
        hoverinfo='skip'
        ))

    fig.add_trace(go.Scatter(
        x=df.index,
        y=error_plus.values,
        fill="tonexty",
        mode='lines',
        line=dict(color=color[1], width=0.1),
        showlegend=False,
        hoverinfo='skip'
        ))
    
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df[var],
        mode="lines",
        line=dict(color=color[0], dash=dash),
        showlegend=showlegend,
        name=name,
        hovertemplate = 'InTEM %{x|%Y}: %{y:.0f} Mt yr⁻¹<extra></extra>',
        ))


errorbars(df_ch4, "intem", color_ch4, None, "Methane")
fig.add_trace(go.Scatter(
    x=df_ch4.index,
    y=df_ch4["invent"],
    line=dict(color=color_ch4[0], dash="dot", width=5),
    showlegend=False,
    hovertemplate = 'Inventory %{x|%Y}: %{y:.0f} Mt yr⁻¹<extra></extra>',
    ))

errorbars(df_n2o, "intem", color_n2o, None, "Nitrous oxide")
fig.add_trace(go.Scatter(
    x=df_n2o.index,
    y=df_n2o["invent"],
    line=dict(color=color_n2o[0], dash="dot", width=4),
    showlegend=False,
    hovertemplate = 'Inventory %{x|%Y}: %{y:.0f} Mt yr⁻¹<extra></extra>',
    ))

errorbars(df_fgas["2012-01-01":], "intem", color_fgas, None, "F-gases")
fig.add_trace(go.Scatter(
    x=df_fgas.index,
    y=df_fgas["invent"],
    line=dict(color=color_fgas[0], dash="dot", width=4),
    showlegend=False,
    hovertemplate = 'Inventory %{x|%Y}: %{y:.0f} Mt yr⁻¹<extra></extra>',
    ))

# Add fake line for InTEM
fig.add_trace(go.Scatter(
    x=[pd.Timestamp("1900-01-01"), pd.Timestamp("1900-01-02")],
    y=[0., 0.],
    line=dict(color="black", dash=None),
    marker=dict(opacity=0., size=0),
    showlegend=True,
    name="Atmospheric data"
    ))

# Add fake line for inventory legend
fig.add_trace(go.Scatter(
    x=[pd.Timestamp("1900-01-01"), pd.Timestamp("1900-01-02")],
    y=[0., 0.],
    line=dict(color="black", dash="dot", width=4),
    marker=dict(opacity=0., size=0),
    showlegend=True,
    name=f"Inventory ({intem_year})"
    ))

fig.update_layout(
    yaxis_title="CO<sub>2</sub>-equivalent emissions (Mt yr⁻¹)",
    template="none",
    autosize=False,
    width=1000,
    height=600,
    legend=dict(
        yanchor="top",
        y=0.77,
        xanchor="right",
        x=0.99,
        traceorder="normal"),
    margin=dict(l=100, r=50, t=50, b=50),
)

fig.layout.font.size=20
fig.layout.font.family="Arial"

fig.update_xaxes(range=[pd.Timestamp("1990-01-01"),
                        pd.Timestamp(f"{intem_year+1}-01-01")])

fig.update_yaxes(range=[0,
                        153])

# Add UKMO logo
fig.add_layout_image(
    dict(
        source='data:image/png;base64,{}'.format(mo_logo.decode()),
        xref="x domain",
        yref="y domain",
        x=1.015, y=0.9,
        sizex=0.25,
        sizey=0.3,
        xanchor="right", yanchor="top"
    )
)

# Add UoB logo
fig.add_layout_image(
    dict(
        source='data:image/png;base64,{}'.format(uob_logo.decode()),
        xref="x domain",
        yref="y domain",
        x=0.99, y=0.99,
        sizex=0.2,
        sizey=0.3,
        xanchor="right", yanchor="top"
    )
)

# Add shading to 2-site period
fig.add_vrect(x0="1990-01-01", x1="2011-06-01", 
              #annotation_text=None, annotation_position="left",
              fillcolor="grey", opacity=0.1, line_width=0)

# Text annotation denoting period
fig.add_annotation(
    xref="x",
    yref="y",
    x="2014-01-01",
    y=45,
    align="left",
    text="DECC network",
    showarrow=False,
    font=dict(
        family="Arial",
        size=15,
        color="Grey",
        )
)
fig.add_annotation(
    xref="x",
    yref="y",
    x="2011-06-01",
    arrowwidth=2,
    arrowcolor="grey",
    align="right",
    y=40,
    ax=150,
    ay=0,
    arrowside="start",
    text=None
)

# Text annotation denoting 2-site period
fig.add_annotation(
    xref="x",
    yref="y",
    x="2008-09-01",
    y=40,
    align="right",
    text="Mace Head, Cabauw",
    showarrow=False,
    font=dict(
        family="Arial",
        size=15,
        color="Grey",
        )
)
fig.add_annotation(
    xref="x",
    yref="y",
    x="2011-06-01",
    arrowwidth=2,
    arrowcolor="grey",
    align="right",
    y=35,
    ax=-150,
    ay=0,
    arrowside="start",
    text=None
)


fig.write_image(f"figures/InTEM_CO2e_{intem_year}.pdf")
fig.write_image(f"figures/InTEM_CO2e_{intem_year}.png")
fig.write_image(f"figures/InTEM_CO2e_{intem_year}.svg")
fig.write_html(f"figures/InTEM_CO2e_{intem_year}.html")

fig.show()

# Zoomed in version (needs updating)

In [ ]:
alpha = 0.6
color_ch4 = ["rgba(86, 119, 194, 1)", f"rgba(86, 119, 194, {alpha})"]
color_n2o = ["rgba(99, 182, 137, 1)", f"rgba(99, 182, 137, {alpha})"]
color_fgas = ["rgba(187, 103, 93, 1)", f"rgba(187, 103, 93, {alpha})"]

fig = go.Figure()

mo_logo = base64.b64encode(open("metoffice_logo.png", 'rb').read())
uob_logo = base64.b64encode(open("uob_logo.png", 'rb').read())

def errorbars(df, var, color, dash, name):

    error_minus=df[var] - df[f"{var}_std"]
    error_plus=df[var] + df[f"{var}_std"]

    fig.add_trace(go.Scatter(
        x=df.index,
        y=error_minus.values,
        fill=None,
        mode='lines',
        line=dict(color=color[1], width=0.1),
        showlegend=False
        ))

    fig.add_trace(go.Scatter(
        x=df.index,
        y=error_plus.values,
        fill="tonexty",
        mode='lines',
        line=dict(color=color[1], width=0.1),
        showlegend=False
        ))
    
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df[var],
        line=dict(color=color[0], dash=dash),
        #showlegend=False,
        name=name
        ))


errorbars(df_ch4, "InTEM", color_ch4, None, "CH<sub>4</sub>")
fig.add_trace(go.Scatter(
    x=df_ch4.index,
    y=df_ch4["Invent"],
    line=dict(color=color_ch4[0], dash="dot", width=5),
    showlegend=False,
    #name="CH<sub>4</sub>"
    ))

errorbars(df_n2o, "InTEM", color_n2o, None, "N<sub>2</sub>O")
fig.add_trace(go.Scatter(
    x=df_n2o.index,
    y=df_n2o["Invent"],
    line=dict(color=color_n2o[0], dash="dot", width=5),
    showlegend=False,
    #name="N<sub>2</sub>O"
    ))

errorbars(df_fgas, "InTEM", color_fgas, None, "F-gases")
fig.add_trace(go.Scatter(
    x=df_fgas.index,
    y=df_fgas["Invent"],
    line=dict(color=color_fgas[0], dash="dot", width=5),
    showlegend=False,
    #name="F-gases"
    ))

# Add fake line for InTEM
fig.add_trace(go.Scatter(
    x=[pd.Timestamp("1999-01-01"), pd.Timestamp("1999-01-02")],
    y=[0., 0.],
    line=dict(color="black", dash=None),
    showlegend=True,
    name="Atmospheric data"
    ))

# Add fake line for inventory legend
fig.add_trace(go.Scatter(
    x=[pd.Timestamp("1999-01-01"), pd.Timestamp("1999-01-02")],
    y=[0., 0.],
    line=dict(color="black", dash="dot", width=5),
    showlegend=True,
    name="Inventory (2021)"
    ))


fig.update_layout(
    # title={"text": "UK non-CO<sub>2</sub> reenhouse gas emissions 2010 - 2020",
    #         "xanchor": "left",
    #         "x": 0.01},
    yaxis_title="CO<sub>2</sub>-equivalent emissions (Gt yr⁻¹)",
    template="none",
    autosize=False,
    width=800,
    height=600,
    legend=dict(
        yanchor="top",
        y=0.7,
        xanchor="right",
        x=0.5),
    margin=dict(l=100, r=50, t=50, b=50),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

fig.update_xaxes(range=[pd.Timestamp("2012-01-01"),
                        pd.Timestamp("2020-01-01")])

fig.update_yaxes(range=[0,
                        80])

# Add UKMO logo
fig.add_layout_image(
    dict(
        source='data:image/png;base64,{}'.format(mo_logo.decode()),
        xref="x domain",
        yref="y domain",
        x=0.01, y=0.01,
        sizex=0.2,
        sizey=0.3,
        xanchor="left", yanchor="bottom"
    )
)

# Add UoB logo
fig.add_layout_image(
    dict(
        source='data:image/png;base64,{}'.format(uob_logo.decode()),
        xref="x domain",
        yref="y domain",
        x=0.23, y=0.025,
        sizex=0.15,
        sizey=0.3,
        xanchor="left", yanchor="bottom"
    )
)

fig.layout.font.size=20
fig.layout.font.family="Arial"

# fig.write_image("InTEM_CO2e_2021.pdf")
# fig.write_image("InTEM_CO2e_2021.png")
# fig.write_html("InTEM_CO2e_2021.html")

fig.show()